# Рекомендация тарифов

### Введение  

В нашем распоряжении данные о поведении клиентов, которые уже перешли на рекомендуемые тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже сделана.  
Построим модель с максимально большим значением *accuracy*, по крайней мере не менее 0.75. Проверим *accuracy* на тестовой выборке.  

Описание данных  

•	сalls — количество звонков,  
•	minutes — суммарная длительность звонков в минутах,  
•	messages — количество sms-сообщений,  
•	mb_used — израсходованный интернет-трафик в Мб,  
•	is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.  

Краткий план работы:
1. Изучение общей информации о базе данных.  
2. Разделение данных на выборки.  
3. Исследование моделей.  
4. Проверка модели на тестовой выборке.  
5. Проверка модели на адекватность.
6. Общий вывод.

### 1. Изучение общей информации о базе данных.

In [1]:
# импорт библиотеки pandas
import pandas as pd
import numpy as np

In [2]:
# чтение файла базы данных
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.duplicated().sum()

0

#### Вывод по изучению базы данных   

Действительно, как и указано в задании, предобработки данных не требуется.

### 2. Разделение данных на выборки

Т.к. тестовой выборки нет, то разделим датасет на три части - обучающую, валидационную и тестовую выборки - в соотношении 3:1:1.  
Будем использовать функцию деления train_test_split библиотеки sklearn. Эта функция позволяет делить выборку только на две части. Поэтому воспользуемся функцией деления в два этапа - сначала 4:1, а затем первую часть разделим в соотношении 3:1.

In [6]:
# импорт функции train_test_split библиотеки sklearn
from sklearn.model_selection import train_test_split

In [7]:
# выделение тестовой выборки
df_1, df_test = train_test_split(df, test_size=0.2, random_state=12345)

In [8]:
# выделение обучающей и валидационной выборок
df_train, df_valid = train_test_split(df_1, test_size=0.25, random_state=12345)

#### Вывод информации о длине полученных выборок

In [9]:
print('Длина выборки')
print('- обучающая:   ', len(df_train))
print('- валидационная:   ', len(df_valid))
print('- тестовая:   ', len(df_test))

Длина выборки
- обучающая:    1928
- валидационная:    643
- тестовая:    643


### 3. Исследование моделей

Исследуем модели, созданные на основе трех разных алгоритмов, - дерево решений, случайный лес и логистическая регрессия. Для дерева решений и случайного леса проверим результат при разных значениях гиперпараметров. Результаты исследования выведем в виде таблицы. Критерием для выбора лучшей модели будем считать долю правильных ответов (accuracy), которую вычислим с помощью функции accuracy_score() библиотеки sklearn.

In [10]:
# импорт необходимых функций и библиотек
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

In [11]:
# переменные для признаков и целевого признака
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [12]:
# заготовка параметров таблицы результатов исследования моделей
columns = ['method', 'parameter', 'accuracy']
data=[]

In [13]:
# модель "решающее дерево" (oграничим глубину дерева max_depth=4, т.к. в датасете только 4 вводных признака)
for md in range (2,5):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=md)
    model_tree.fit(features_train, target_train)
    predicted_valid_tree = model_tree.predict(features_valid)
    acc_tree = accuracy_score(target_valid, predicted_valid_tree)
    line_tree=['decision_tree',md,acc_tree]
    data.append(line_tree)
summary=pd.DataFrame(data=data, columns=columns)

In [14]:
# модель "случайный лес" 
# переберем значения n_estimators от 10 до 50, возьмем только кратные десяти,
# oграничим макс глубину деревьев: max_depth=4, т.к. в датасете только 4 вводных признака
for estim in range(10, 51, 10):
    model_forest = RandomForestClassifier(n_estimators=estim, max_depth=4, random_state=12345)
    model_forest.fit(features_train, target_train)
    predicted_valid_forest = model_forest.predict(features_valid)
    acc_forest = accuracy_score(target_valid, predicted_valid_forest)
    summary=summary.append(pd.Series(['random_forest',estim,acc_forest],index = summary.columns), ignore_index=True)

In [15]:
# модель "логистическая регрессия"
model_lr = LogisticRegression(random_state=12345, solver='lbfgs')
model_lr.fit(features_train, target_train)
predicted_valid_lr = model_lr.predict(features_valid)
acc_lr = accuracy_score(target_valid, predicted_valid_lr)
summary=summary.append(pd.Series(['logistic regression',np.nan, acc_lr],index = summary.columns), ignore_index=True)

In [16]:
# результаты исследования в сравнении
summary

,method,parameter,accuracy
0,decision_tree,2.0,0.757387
1,decision_tree,3.0,0.765163
2,decision_tree,4.0,0.763608
3,random_forest,10.0,0.777605
4,random_forest,20.0,0.776050
5,random_forest,30.0,0.776050
6,random_forest,40.0,0.779160
7,random_forest,50.0,0.774495
8,logistic regression,NaN,0.726283


In [17]:
# выбор лучшей модели
best=summary[summary['accuracy']==summary['accuracy'].max()].reset_index(drop=True)
best

,method,parameter,accuracy
0,random_forest,40.0,0.77916


Обучим выбранную модель на исходном датасете за исключением тестовой выборки: чем больше данных, тем лучше результат.
Валидационная выборка теперь не нужна, т.к. мы уже определились с лучшей моделью.

In [20]:
features = df_1.drop(['is_ultra'], axis=1)
target = df_1['is_ultra']
model = RandomForestClassifier(n_estimators=40, max_depth=4, random_state=12345)
model.fit(features, target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

#### Вывод по исследованию моделей

Наилучший результат показала модель "случайный лес" со значением гиперпараметра n_estimators == 40.  
Доля правильных ответов этой модели на валидационной выборке составила 0.779.

### 4. Проверка модели на тестовой выборке

In [21]:
# переменные для признаков и целевого признака
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [22]:
# предсказания для тестовых данных и доля правильных ответов
predicted_test = model.predict(features_test)
acc_test = accuracy_score(target_test, predicted_test).round(3)

In [23]:
# подготовка вывода результата на экран
result=best.rename(columns={'parameter':'n_estimators','accuracy':'test_accuracy'})
result['test_accuracy']=acc_test

In [24]:
# вывод результата
result

,method,n_estimators,test_accuracy
0,random_forest,40.0,0.793


#### Вывод по проверке модели на тестовой выборке

Т.о., модель "случайный лес" со значением гиперпараметра n_estimators = 40 на тестовой выборке обеспечила долю правильных ответов 0.793.  
Т.е. модель удовлетворяет поставленному условию accuracy > 0.75.

### 5. Проверка модели на адекватность

Для проверки модели на адекватность сравним ее результат с результатом работы функции dummy библиотеки sklearn.  
Функцию dummy применим с разными значениями параметра strategy, т.е. модель будет строиться по разным алгоритмам.  
Модели будем обучать на датасете df_1 (исходном датасете за исключением тестовой выборки), проверять - на тестовой выборке df_test.  
Будем считать, что проверка на адекватность пройдена, если проверяемая модель покажет лучший результат, чем любая из моделей dummy.

In [25]:
# импорт функции dummy
from sklearn.dummy import DummyClassifier

In [26]:
# список значений параметра strategy
strategies = ['constant', 'uniform', 'stratified', 'most_frequent', 'prior']

In [27]:
# создание и обучение моделей dummy, сравнение с проверяемой моделью 
print('Доля правильных ответов:')
sanity_check_counter = 0 # счетчик количества результатов, полученных dummy, превосходящих результат проверяемой модели
for strat in strategies:
    if strat == 'constant': # данное значение выведено отдельно, т.к. требуется указать дополнительный параметр (constant)
        model_dummy = DummyClassifier(strategy='constant', constant=1, random_state=12345) # создание модели dummy
    else:
        model_dummy = DummyClassifier(strategy=strat, random_state=12345) # создание модели dummy
    model_dummy.fit(features, target) # обучение модели dummy
    predicted_test_dummy = model_dummy.predict(features_test) # работа модели dummy
    acc_test_dummy = accuracy_score(target_test, predicted_test_dummy).round(3) # результат работы модели dummy
    if acc_test_dummy > acc_test: # сравнение результата работы модели dummy с результатом проверяемой модели
        sanity_check_counter +=1
    print('dummy',strat,':  ',acc_test_dummy)

Доля правильных ответов:
dummy constant :   0.305
dummy uniform :   0.49
dummy stratified :   0.577
dummy most_frequent :   0.695
dummy prior :   0.695


In [28]:
# доли тарифов в тестовой выборке
print('Доля тарифа Ультра:  ', round(df_test['is_ultra'].sum()/len(df_test),3))
print('Доля тарифа Смарт:  ', round((1-df_test['is_ultra'].sum()/len(df_test)), 3))

Доля тарифа Ультра:   0.305
Доля тарифа Смарт:   0.695


Как можно заметить, модели dummy дают результат либо близкий к случайному (50%), либо совпадающий с распределением значений целевого признака в тестовой выборке.

#### Вывод по проверке модели на адекватность

In [29]:
if sanity_check_counter == 0: # т.е., если ни одна из моделей dummy не превзошла проверяемую модель, то:
    print ('Модель адекватна.')
else: # в ином случае:
    print ('Модель неадекватна.')

Модель адекватна.


### 6. Общий вывод

В ходе работы были проделаны следующие операции и получены такие результаты:  
Разделили датасет на три части - обучающую, валидационную и тестовую выборки - в соотношении 3:1:1 с использованием функции train_test_split библиотеки sklearn.  
Провели сравнение моделей, созданных на основе трех разных алгоритмов, - дерево решений, случайный лес и логистическая регрессия. Для дерева решений и случайного леса применили разные значения гиперпараметров. За критерий выбора лучшей модели приняли долю правильных ответов (accuracy), которую вычислили с помощью функции accuracy_score() библиотеки sklearn.  
**Наилучший результат показала модель "случайный лес" со значением гиперпараметра n_estimators = 40.**  
На тестовой выборке эта модель дала долю правильных ответов 0.793, что удовлетворяет поставленному условию accuracy > 0.75.  
Выбранная модель успешно прошла проверку на адекватность - сравнение с результатами работы моделей, полученных с помощью функции dummy библиотеки sklearn.